In [7]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

# Thêm thư mục gốc vào đường dẫn hệ thống để import được 'src'
sys.path.append(os.path.abspath('..'))

# Import các module "nhà làm" (Code bạn đã đóng gói)
from src.dataloader.loader import get_data_loaders
from src.models.hybrid_model import BrainTumorModel
from src.utils.helpers import init_weights

# Setup thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

Đang sử dụng thiết bị: cpu


In [8]:
# 1. Load dữ liệu
# Lưu ý: Đường dẫn data phải trỏ đúng ra ngoài thư mục notebooks
data_dir = '../data/kaggle' 

try:
    train_loader, val_loader, test_loader, class_names = get_data_loaders(data_dir, batch_size=32)
    print(f"✅ Load dữ liệu thành công! Các lớp: {class_names}")
except FileNotFoundError:
    print("❌ Không tìm thấy dữ liệu. Hãy kiểm tra lại đường dẫn data_dir!")

# 2. Khởi tạo Model
model = BrainTumorModel(num_classes=4).to(device)

# 3. Khởi tạo trọng số (Bước quan trọng để tránh lỗi model không học)
model.apply(init_weights)
print("✅ Đã khởi tạo Model và Trọng số!")

✅ Load dữ liệu thành công! Các lớp: ['glioma', 'meningioma', 'notumor', 'pituitary']
✅ Đã khởi tạo Model và Trọng số!


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

def run_epoch(model, loader, criterion, optimizer=None, is_train=True):
    if is_train:
        model.train()
    else:
        model.eval()
        
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Dùng torch.set_grad_enabled để bật/tắt gradient tự động
    with torch.set_grad_enabled(is_train):
        for images, labels in tqdm(loader, leave=False, desc="Training" if is_train else "Validating"):
            images, labels = images.to(device), labels.to(device)
            
            if is_train:
                optimizer.zero_grad()
                
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            if is_train:
                loss.backward()
                optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return running_loss / len(loader), 100 * correct / total

In [ ]:
NUM_EPOCHS = 30
PATIENCE = 8
best_val_acc = 0.0
patience_counter = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("🚀 Bắt đầu Training...")

for epoch in range(NUM_EPOCHS):
    # Train
    train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, is_train=True)
    
    # Validate
    val_loss, val_acc = run_epoch(model, val_loader, criterion, is_train=False)
    
    # Scheduler step
    scheduler.step(val_loss)
    
    # Lưu history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.4f} Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} Acc: {val_acc:.2f}%")
    
    # Early Stopping & Save
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        # Lưu model ra thư mục ngoài để dễ quản lý
        torch.save(model.state_dict(), '../best_model_TAndE.pth')
        print("🔥 Đã lưu model tốt nhất!")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("🛑 Early Stopping!")
            break

🚀 Bắt đầu Training...
